In [ ]:
import time 
import datetime
from datetime import timedelta
import json
import redis 
import bz2
import requests
import json
from threading import Thread
import pymongo
import dns
import sys
import urllib3

#no warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#auth Redis
RedisSubscriber = redis.Redis(host='localhost', port=6379,db=0)
RedisSubscriber.flushdb()
redis_key = 0.0

#auth Twitter
consumer_key,consumer_secret = open('keys.txt','r').readlines()
consumer_key = consumer_key[:-1]

In [ ]:
def get_bearer_token(key, secret):
    response = requests.post(
        'https://api.twitter.com/oauth2/token',
        auth=(key, secret),
        data={'grant_type': 'client_credentials'},
        headers={'User-Agent': 'mmazzola'})

    if response.status_code != 200:
        raise Exception(f'Cannot get a Bearer token (HTTP %d): %s' % (response.status_code, response.text))
    body = response.json()
    return body['access_token']

def save2redis(item):
    #try redis pipeline (anche se non so quanto sia utile una cosa del genere)
    global redis_key
    RedisSubscriber.set(redis_key,bz2.compress(item).encode("utf-8"))
    redis_key+=0.01
    
def preprocess(response_line):
    '''Saving a fixed version of a tweet, with:
            - status id and user id
            - when it is created
            - user and status reply (if one)
            - lang 
            - text
            - id of the retweet (if one)
    '''
    
    json_ = json.loads(response_line)
    dicts = {}
    attrs = ['id','created_at','in_reply_to_user_id','in_reply_to_status_id','lang']
    for i in attrs:
        dicts[i] = json_[i]
    
    if(json_['truncated'] == True):
        dicts['text'] = json_['extended_tweet']['full_text']
    else:
        dicts['text'] = json_['text']
        
    if("retweeted_status" in json_.keys()):
        dicts['retweeted'] = json_['retweeted_status']['id']
    else:
        dicts['retweeted'] = None
        
    dicts['user'] = json_['user']['id']
    return json.dumps(dicts) #return string

def stream_connect(partition,all_=False):
    response = requests.get('https://api.twitter.com/labs/1/tweets/stream/covid19?partition={}'.format(partition),
                            headers={'User-Agent': 'mmazzola',
                                     'Authorization': 'Bearer {}'.format(get_bearer_token(consumer_key, consumer_secret))},
                            stream=True,verify=False)
    
    for response_line in response.iter_lines():
        if response_line:
            if all_==True:
                #get everything
                save2redis(json.dumps(json.loads(response_line)))
            else:
                try:
                    #get only relevant feature (save memory)
                    dicts_preprocessed = preprocess(response_line)
                    save2redis(dicts_preprocessed)
                except:
                    #if problems, get everything
                    save2redis(response_line)
def main():
    print('### BEGINNING ###')
    while(True):
        for partition in range(1, 5):
            x = Thread(target=stream_connect, args=(partition,)).start()
        time.sleep(30)
    print("### END ###")
    
if __name__ == '__main__':
    main()